In [1]:
from typing import Tuple, List
from AcasXuNet import AcasXu
from ReluPatterns import Patterns
import torch
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import numpy as np
PATH = '/home/nle/workspace/VerifyNNE/datasets/ACAS/acas_nets/ACASXU_run2a_1_3_batch_2000.nnet'
network = AcasXu(PATH)
print(network.means)
print("min", network.mins)
print("max", network.maxs)
dataset: Tuple[Tensor] = torch.load('/home/nle/workspace/VerifyNNE/datasets/ACAS/acas_nets/AcasNetID<1,3>-normed-train.pt')

dataset_train = TensorDataset(dataset[0], dataset[1])
print(dataset[0].shape, dataset[1].shape)
print(dataset_train)

loader = DataLoader(dataset_train, batch_size=dataset[0].shape[0], shuffle=True)

all_patterns: Patterns = Patterns(model = network,
                        dataloader = loader,
                        labels = range(5),
                        layers = ['layer0', 'layer1', 'layer2', 'layer3', 'layer4'])

print(all_patterns)
all_preds: List[int] = []
true_labels: List[int] = []
network.register_log()
for inputs, outputs in loader:
    raw_pred = network(inputs)
    pred: List[int] = torch.argmin(raw_pred, dim = 1).squeeze().tolist()
    all_preds.extend(pred)
    true_labels.extend(outputs.squeeze().tolist())

print(classification_report(true_labels, all_preds))


[19791.091, 0.0, 0.0, 650.0, 600.0, 7.5188840201005975]
min [0.0, -3.141593, -3.141593, 100.0, 0.0]
max [60760.0, 3.141593, 3.141593, 1200.0, 1200.0]
torch.Size([10000, 5]) torch.Size([10000])
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000
           1       1.00      1.00      1.00      2000
           2       1.00      1.00      1.00      2000
           3       1.00      1.00      1.00      2000
           4       1.00      1.00      1.00      2000

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [2]:
EPSILON = 0
alpha_patterns = {}
for label in all_patterns.label2patterns:
    patterns = all_patterns.label2patterns[label]
    print(patterns.shape)


    relu_sum = np.sum(patterns, axis = 0).squeeze()

    stable_idx = np.concatenate([np.where(relu_sum<=EPSILON*patterns.shape[0]), 
                                    np.where(relu_sum>=(1-EPSILON)*patterns.shape[0])],
                                axis = 1
                                ).squeeze()
    print(stable_idx)

(2000, 250)
[  2  20  27  30  43  59  61  63  65  69  73  87  99 104 132 141 145 152
 153 164 165 175 176 178 182 185 186 188 201 205 207 208 211 220 221 223
 226   1  29  96 106 148 166 174 179 192 232 243]
(2000, 250)
[  2  27  30  59  63  65  69  73  87  99 104 132 160 165 170 175 186 188
 207 208 211 220 223 237 238 245  29 106 148 179 232 243]
(2000, 250)
[  2  27  30  63  65  69  73  87  99 104 132 165 176 186 188 205 207 208
 211  29 148 179 232 243]
(2000, 250)
[  2  27  30  63  65  69  73  87  99 104 132 165 188 207 208 220 245  29
 148 179 232]
(2000, 250)
[  2  27  30  63  65  69  73  87  99 104 132 165 188 207 208  29 148 179
 232]
